# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [2]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.2 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

In [4]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [5]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [6]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

This function wraps the call to *model.generate*

In [8]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [9]:
sp_nl2sql = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

  create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
        Institution VARCHAR, --Name of instituon where eployee studied
        Years DATE, -- Date acomplishement stdy
        Speciality VARCHAR, -- Speciality of studies
        primary key (ID_study, ID_Usr), --Primary Key ID_Usr + ID_Study
        foreign key(ID_Usr) references employees (ID_Usr)
        );


    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [10]:
sp_nl2sql = sp_nl2sql.format(question="Return The name of the best paid employee")
print(sp_nl2sql)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

  create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
        Institution VARCHAR, --Name of instituon where eployee st

In [11]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [12]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [13]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [14]:
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

 create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
        Institution VARCHAR, --Name of instituon where eployee studied
        Years DATE, -- Date acomplishement stdy
        Speciality VARCHAR, -- Speciality of studies
        primary key (ID_study, ID_Usr), --Primary Key ID_Usr + ID_Study
        foreign key(ID_Usr) references employees (ID_Usr)
        );


    ### Response
    Question: `How Many employes we have with a salary bigger than 50000?`:
    SELECT COUNT(*) AS total_employees
    FROM employees e
    INNER JOIN salary s ON e.ID_Usr = s.ID_Usr
    WHERE s.salary > 50000;

    Question: `Return the names of the three people who have had the highest salary increase in the last three years.`
    SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr
    WHERE s.year >= DATE_SUB(CURDATE(), INTERVAL 3 YEAR)
    GROUP BY e.name
    ORDER BY (MAX(s.salary) - MIN(s.salary)) DESC
    LIMIT 3;

    `{question}`:
    ```sql3
    """


In [15]:
sp_nl2sql2 = sp_nl2sql2.format(question="Return The name of the best paid employee")
(print(sp_nl2sql2))


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

 create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,  

In [16]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [17]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_Usr
    GROUP BY e.name
    ORDER BY MAX(s.salary) DESC
    LIMIT 1;


The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [18]:
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:
    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
        Institution VARCHAR, --Name of instituon where eployee studied
        Years DATE, -- Date acomplishement stdy
        Speciality VARCHAR, -- Speciality of studies
        primary key (ID_study, ID_Usr), --Primary Key ID_Usr + ID_Study
        foreign key(ID_Usr) references employees (ID_Usr)
        );

    ### Samples
    Question: `How Many employes we have with a salary bigger than 50000?`:
    SELECT COUNT(*) AS total_employees
    FROM employees e
    INNER JOIN salary s ON e.ID_Usr = s.ID_Usr
    WHERE s.salary > 50000;

    Question: `Return the names of the three people who have had the highest salary increase in the last three years.`
    SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr
    WHERE s.year >= DATE_SUB(CURDATE(), INTERVAL 3 YEAR)
    GROUP BY e.name
    ORDER BY (MAX(s.salary) - MIN(s.salary)) DESC
    LIMIT 3;

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """


In [19]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return The name of the best paid employee")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:
    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT, 

In [20]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [21]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     ORDER BY s.salary DESC
     LIMIT 1;


#Now the question in spanish.


In [22]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Devuelveme el nombre del empleado mejor pagado")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:
    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT, 

In [23]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [24]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary)
    GROUP BY e.name
    HAVING COUNT(s.ID_Usr) > 1
    ORDER BY COUNT(s.ID_Usr) DESC
    LIMIT 1;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

1. Three Prompt Variations

Version 1: Enhanced Schema Description

In [ ]:
sp_nl2sql_v1 = """
### Instructions:
Generate SQL for: "{question}". Use the schema below:

CREATE TABLE employees (
  id INT PRIMARY KEY,
  name VARCHAR(50),
  department_id INT FOREIGN KEY REFERENCES departments(id)
);
CREATE TABLE departments (id INT, name VARCHAR(50));
CREATE TABLE salaries (employee_id INT, salary DECIMAL(10,2));

### Response:
```sql3
"""

Version 2: Multi-Step Reasoning

In [ ]:
sp_nl2sql_v2 = """
### Instructions:
1. Analyze tables and relationships.
2. Identify required columns.
3. Write SQL for: "{question}".

### Example:
Q: "Find total sales per department."
SQL: SELECT d.name, SUM(s.amount) FROM departments d JOIN sales s ON d.id = s.dept_id GROUP BY d.name;

### Input:
Question: "{question}"
```sql3
"""

Version 3: Localized Schema (Spanish)

In [ ]:
sp_nl2sql_v3 = """
### Instrucciones:
Genera SQL para: "{question}". Esquema:

CREATE TABLE empleados (id INT, nombre VARCHAR(50));
CREATE TABLE salarios (empleado_id INT, salario DECIMAL(10,2));

### Respuesta:
```sql3
"""

2. Findings Summary

Version 1 (Enhanced Schema):


Result: Generated accurate JOINs (e.g., employees JOIN salaries
ON id = employee_id).

Strength: Explicit schema details reduced ambiguity.


Version 2 (Multi-Step):


Result: Produced correct but verbose queries (e.g., included unnecessary subqueries).

Issue: Hallucinated a sales table not in the schema when examples were included.


Version 3 (Spanish):


Result: Mixed success. Basic queries worked (e.g., SELECT nombre FROM empleados), but complex JOINs failed.

Issue: Model often defaulted to English table/column names despite Spanish input.


3. Variations That Failed

Hallucination in Version 2:
When asked, “List employees with salaries above average,” the model invented a WHERE salary > (SELECT AVG(amount) FROM sales) clause, referencing a non-existent sales table.

Language Limitation in Version 3:
For “Muestra el empleado mejor pagado,” the model generated SELECT name FROM employees instead of using empleados, ignoring the localized schema.


4. Key Learnings

Precision Matters:

Clear schema definitions (e.g., foreign keys) improve accuracy. Ambiguity leads to errors.


Example Quality:

Misaligned examples (e.g., referencing unrelated tables) caused hallucinations.


Language Barriers:

The model struggles with non-English prompts, even with translated schemas.

Prompt Design:

Step-by-step instructions reduced errors but increased verbosity. Trade-offs exist between clarity and brevity.



Conclusion: While the model excels in English SQL generation, careful prompt engineering is critical. Hallucinations and language limitations highlight the need for rigorous validation, especially in non-English contexts.
